In [1]:
import torch
import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm
from transformers import AutoImageProcessor, ViTMAEModel
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, balanced_accuracy_score, matthews_corrcoef

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model_name = "facebook/vit-mae-base"

image_processor = AutoImageProcessor.from_pretrained(model_name, use_fast=True)

model = ViTMAEModel.from_pretrained(model_name).to(device)


In [3]:
df_allCarabids = pd.read_csv("/fs/ess/PAS2136/Rayeed/Carabids-100K-V2/AllCarabids.csv", sep="\t")
df_insect1M = df_allCarabids[df_allCarabids["filepath"].str.startswith("Insect-1M")].copy()
df_insect1M.rename(columns={"filepath": "ImageFilePath", "class": "ScientificName"}, inplace=True)
cols = ['ImageFilePath', 'ScientificName']
df = df_insect1M[cols]
df['ImageFilePath'] = "/fs/ess/PAS2136/Rayeed/Carabids-100K-V2/" + df['ImageFilePath']
df = df.fillna("Unknown")
df.head(2)

/tmp/slurmtmp.826913/ipykernel_2193381/1858217642.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ImageFilePath'] = "/fs/ess/PAS2136/Rayeed/Carabids-100K-V2/" + df['ImageFilePath']


,ImageFilePath,ScientificName
0,/fs/ess/PAS2136/Rayeed/Carabids-100K-V2/Insect...,Cicindela sexguttata
1,/fs/ess/PAS2136/Rayeed/Carabids-100K-V2/Insect...,Cicindela sexguttata


In [4]:
def extract_features(image_path) :
    
    image = Image.open(image_path).convert("RGB")

    inputs = image_processor(images=image, return_tensors="pt").to(device)
    
    with torch.no_grad():
        features = model(**inputs).last_hidden_state.mean(dim=1)
    
    return features.cpu().numpy()

In [5]:
df_known = df[df["ScientificName"] != "Unknown"]

df_unknown = df[df["ScientificName"] == "Unknown"]


X_known = np.vstack([extract_features(img) for img in tqdm(df_known["ImageFilePath"], desc="Processing ...")])


le = LabelEncoder()

y_known = le.fit_transform(df_known["ScientificName"])

df_indices_known = df_known.index


X_train, X_test_known, y_train, y_test_known, train_idx, test_idx_known = train_test_split(
    X_known, y_known, df_indices_known, test_size=0.2, random_state=42
)


X_test_unknown = np.vstack([extract_features(img) for img in tqdm(df_unknown["ImageFilePath"], desc="Processing Unknown ...")])

test_idx_unknown = df_unknown.index

y_test_unknown = np.full(len(X_test_unknown), -1)


X_test = np.vstack([X_test_known, X_test_unknown])

y_test = np.concatenate([y_test_known, y_test_unknown])

test_idx = np.concatenate([test_idx_known, test_idx_unknown])

print(f"Train set: {X_train.shape}, Test set: {X_test.shape}")



test_df = df.loc[test_idx].copy()

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)

X_test_scaled = scaler.transform(X_test)



Processing Unknown ...: 100%|██████████| 4328/4328 [02:12<00:00, 32.69it/s]


Train set: (16222, 768), Test set: (8384, 768)


In [6]:
seed = 99

models = {
    "NaiveBayes": GaussianNB(),
    "LogisticRegression": LogisticRegression(max_iter=100),
    "NearestNeighbor": KNeighborsClassifier(n_neighbors=5),    
    "MLP_Baseline": MLPClassifier(hidden_layer_sizes=(100,), max_iter=300, random_state=seed)
}

predictions = {}

metrics = {}

for name, model in models.items():
    
    model.fit(X_train_scaled, y_train)
    preds = model.predict(X_test_scaled)
    predictions[name] = preds
    
    acc = accuracy_score(y_test, preds)
    prec = precision_score(y_test, preds, average="weighted")
    rec = recall_score(y_test, preds, average="weighted")
    f1 = f1_score(y_test, preds, average="weighted")
    bal_acc = balanced_accuracy_score(y_test, preds)
    mcc = matthews_corrcoef(y_test, preds)
    
    metrics[name] = {"Model": name, "Accuracy": acc, "Precision": prec, "Recall": rec, "F1-Score": f1, "Balanced Acc": bal_acc, "MCC": mcc}
    print(f"{name:<25} | Acc: {acc:.2%} | Prec: {prec:.2%} | Rec: {rec:.2%} | F1: {f1:.2%} | Bal Acc: {bal_acc:.2%} | MCC: {mcc:.4f}")


metrics_df = pd.DataFrame(metrics).T

MLP_Baseline              | Acc: 9.02% | Prec: 5.63% | Rec: 9.02% | F1: 6.44% | Bal Acc: 10.68% | MCC: 0.1039


/users/PAS2136/rayees/myenv/lib64/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/users/PAS2136/rayees/myenv/lib64/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/users/PAS2136/rayees/myenv/lib64/python3.9/site-packages/sklearn/metrics/_classification.py:2480: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


In [7]:
test_df = test_df.assign(**{f"Pred_{name}": le.inverse_transform(pred) for name, pred in predictions.items()})
test_df.head(2)

,ImageFilePath,ScientificName,Pred_NaiveBayes,Pred_LogisticRegression,Pred_NearestNeighbor,Pred_MLP_Baseline
8222,/fs/ess/PAS2136/Rayeed/Carabids-100K-V2/Insect...,Cicindela willistoni,Ellipsoptera macra,Cicindela repanda,Cicindela duodecimguttata,Cicindela repanda
15309,/fs/ess/PAS2136/Rayeed/Carabids-100K-V2/Insect...,Cicindela hirticollis,Cicindela hirticollis,Cicindela hirticollis,Cicindela hirticollis,Cicindela hirticollis


In [8]:
metrics_df

,Model,Accuracy,Precision,Recall,F1-Score,Balanced Acc,MCC
NaiveBayes,NaiveBayes,0.04282,0.03156,0.04282,0.028655,0.056998,0.048788
LogisticRegression,LogisticRegression,0.104127,0.061738,0.104127,0.068152,0.113994,0.119863
NearestNeighbor,NearestNeighbor,0.060472,0.053377,0.060472,0.04741,0.080925,0.06943
MLP_Baseline,MLP_Baseline,0.090172,0.056308,0.090172,0.064422,0.106823,0.103883


In [9]:
test_df.to_csv("/users/PAS2136/rayees/CV4A - Benchmarking/Insect1M/9.ViTMAE-linear-probing-species.csv", index=False)
metrics_df.to_csv("/users/PAS2136/rayees/CV4A - Benchmarking/Insect1M//9.ViTMAE-linear-probing-species-metrics.csv", index=False)